In [1]:
import findspark

findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Failure Prediction on Google Borg Cluster Traces').master('local[*]').getOrCreate()
SparkContext.setSystemProperty('spark.executor.memory', '2g')
SparkContext.setSystemProperty('spark.driver.memory', '2g')

jobs_train_unbalanced_df = spark.read.parquet("./training_data/jobs_data_unbalanced.parquet")
tasks_train_unbalanced_df = spark.read.parquet("./training_data/tasks_data_unbalanced.parquet")

In [2]:
print(jobs_train_unbalanced_df.filter(jobs_train_unbalanced_df.event_success == 0).count())
print(jobs_train_unbalanced_df.filter(jobs_train_unbalanced_df.event_success == 1).count())

print(tasks_train_unbalanced_df.filter(tasks_train_unbalanced_df.event_success == 0).count())
print(tasks_train_unbalanced_df.filter(tasks_train_unbalanced_df.event_success == 1).count())

543968
398916
255802001
13987676


In [4]:
import pandas as pd
from imblearn.over_sampling import SMOTE

pandas_df = jobs_train_unbalanced_df.toPandas()

x = pandas_df.drop('event_success', axis=1)
y = pandas_df['event_success']

smote = SMOTE(random_state=13)
x_resampled, y_resampled = smote.fit_resample(x, y)

jobs_train_unbalanced_df = spark.createDataFrame(pd.concat([x_resampled, y_resampled], axis=1))

C:\Programming\Apache-Spark\spark-3.0.0-bin-hadoop3.2\python\pyspark\sql\pandas\conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [5]:
failed_tasks_df = tasks_train_unbalanced_df.filter(tasks_train_unbalanced_df.event_success == 0)
success_tasks_df = tasks_train_unbalanced_df.filter(tasks_train_unbalanced_df.event_success == 1)

sample_fraction = float(success_tasks_df.count()) / float(failed_tasks_df.count())
sampled_majority = failed_tasks_df.sample(fraction=sample_fraction, seed=13)

tasks_train_unbalanced_df = success_tasks_df.union(sampled_majority)

In [6]:
print(jobs_train_unbalanced_df.filter(jobs_train_unbalanced_df.event_success == 0).count())
print(jobs_train_unbalanced_df.filter(jobs_train_unbalanced_df.event_success == 1).count())

print(tasks_train_unbalanced_df.filter(tasks_train_unbalanced_df.event_success == 0).count())
print(tasks_train_unbalanced_df.filter(tasks_train_unbalanced_df.event_success == 1).count())

543968
543968
13988439
13987676


In [7]:
jobs_train_unbalanced_df.write.parquet("./training_data/jobs_data.parquet")
tasks_train_unbalanced_df.write.parquet("./training_data/tasks_data.parquet")